In [ ]:
%load_ext autoreload
%autoreload 2
from amftrack.pipeline.launching.run_super import (
    run_launcher,
    directory_scratch,
    directory_project,
    directory_project,
    run_parallel_stitch,
    run_parallel_transfer,
)
import os
from amftrack.util.sys import (
    get_dates_datetime,
    get_dirname,
    temp_path,
    get_data_info,
    update_plate_info,
    update_analysis_info,
    get_analysis_info,
    get_current_folders,
    get_folders_by_plate_id,
)

from time import time_ns
from amftrack.util.dbx import upload_folders, load_dbx, download, get_dropbox_folders_prince
from datetime import datetime
from amftrack.pipeline.launching.run_super import (
    run_parallel,
    directory_scratch,
    directory_project,
    run_parallel_stitch,
)
from amftrack.util.dbx import read_saved_dropbox_state,get_dropbox_folders_prince


In [ ]:
# plates = [
# "241_20230403",
# "247_20230422"
# ]
#runnning on 31/05
plates = [
"3_20220426",
"12_20220502",
"13_20220422",
"16_20220419",
"21_20220502",
"480_20221205",
"28_20230227",
"206_20230303",
"202_20230314",
"218_20230227",
"219_20230307",
"229_20230330",
# "240_20230328"
]

In [ ]:
plates = [
# "21_20220502",
# "480_20221205",
"240_20230328",

]

In [ ]:
plates = [
"831_20230310",
"833_20230321",
"834_20230508",
"836_20230327",
"840_20230404",
"841_20230327",
"842_20230320",
"250_20230508",
"845_20230320",
"847_20230406",
"848_20230327",
"849_20230417",
"850_20230511",
"855_20230324",
"856_20230330",
"861_20230411",
"862_20230403",
"863_20230418",
"865_20230409",
"866_20230415"

]
plates = [
"840_20230404",
"850_20230511",
"861_20230411"

]

In [ ]:
plates = ["545_20220115"]

In [ ]:
plates = [
    # "52_20220707",
    # "56_20220701",
    # "59_20201126",
    # "69_20201119",
    # "76_20210108",
    # "81_20220704",
    # "84_20220621",
    # "94_20201123",
    # "102_20201226",
    # "152_20201224",
    "59_20201126",

    "792_20210915",
    "1076_20220511",
    '1014_20220525', 
  '1023_20220502',
  '1042_20220523',
# '1045_20220504',
  '1048_20220506',
  '1053_20220413',
        "52_20220707",
    "56_20220701",
]


In [ ]:
run_parallel_transfer(
    "from_drop.py",
    [directory_targ],
    folders_drop,
    50,
    "24:00:00",
    "staging",
    cpus=1,
    node="staging",
    name_job="kaikai",
)

In [ ]:
all_folders_drop = get_dropbox_folders("/DATA/PRINCE", True)

In [ ]:
dir_drop = "DATA/PRINCE"
all_folders_drop = read_saved_dropbox_state("/DATA/PRINCE")
folders_drop = all_folders_drop.loc[all_folders_drop["unique_id"].isin(plates)]
update_plate_info(directory_targ, local=True, strong_constraint=False)
all_folders = get_current_folders(directory_targ, local=True)
# if len(all_folders) > 0:
folders = all_folders.loc[all_folders["unique_id"].isin(plates)]
folders_drop2 = folders_drop.loc[~folders_drop["folder"].isin(folders["folder"])]
folders_drop3 = folders_drop2.loc[~folders_drop2["folder"].str.contains("Analysis")]

In [ ]:
# all_folders_drop = read_saved_dropbox_state("/DATA/PRINCE")
all_folders_drop.loc[all_folders_drop["Plate"]=="440"]

In [ ]:
# plates= ['94_20201123','102_20201226','76_20210108','152_20201224','792_20210915']
# plates= ['52_20220707','56_20220701','839_20220325','938_20220325','1076_20220511']
plates = [
    "52_20220707",
    "56_20220701",
    "81_20220704",
    "89_20220705",
    "91_20220708",
    "839_20220325",
    "1076_20220511",
]  # full run done!
plates = [
    "84_20220621",
    "877_20220419",
    "938_20220325",
    "941_20220404",
]  # for spore data

plates = [
    "106_20220826",
    "108_20220823",
    "111_20220823",
    "66_20220823",
    "105_20220817",
    "100_20220812",
    "75_20220808",
    "99_20220815",
]

plates = ["94_20201123"]  # for spore data
# plates = ['1048_20220506','84_20220621'] #for spore data
# plates = [
#     "799_20211001",
#     "803_20211012",
#     "809_20211012",
#     "907_20220321",
#     "943_20220322",
#     "403_20221005",
#     "408_20221013",
#     "88_20210915",
#     "790_20210907",
#     "69_20201119",
#     "94_20201123",
#     "59_20201126",
#     "599_20201217",
#     "139_20201214",
#     "40_20201214"
# ]

In [ ]:
plates = ["910_202305016"]

In [ ]:
os.path.join(conda_path,'envs','amftrack')

In [ ]:
plates= ['41_20230221',
 '41_20220310',
 '41_20220310',
 '63_20230510',
 '56_20230511',
 '56_20230511',
 '55_20230510',
 '51_20230510',
 '50_20230511',
 '49_20230510',
 '64_20230508',
 '60_20230508',
 '59_20230508',
 '63_20230510',
 '61_20230510',
 '55_20230510',
 '51_20230510',
 '49_20230510',
 '50_20230511',
 '64_20230508',
 '60_20230508',
 '59_20230508',
 '72_20230501',
 '69_20230501',
 '68_20230501',
 '67_20230501',
 '118_20220426',
 '115_20220324',
 '109_20220324',
 '106_20220324',
 '100_20220324',
 '50_20220310',
 '49_20220310',
 'network_extraction',
 '47_20220310',
 '45_20220310',
 '69_20230501',
 '67_20230501',
 '68_20230501',
 '2_20230306',
 '6_20230221',
 '10_20230227',
 '35_20230227',
 '29_20230221',
 '26_20230308',
 '26_20230308',
 '34_20230227',
 '25_20230221',
 '47_20230221',
 '21_20230306',
 '45_20230221',
 '18_20230221',
 '42_20230221',
 '4_20230221',
 '17_20230221',
 '15_20230221',
 '5_20230306',
 '33_20230221',
 '2_20230306',
 '14_20230301',
 '2_20230306',
 'network_extraction',
 '13_20230221',
 '107_20220426',
 '119_20220426',
 '83_20220426',
 '80_20220426',
 '68_20220426',
 '60_20220310',
 '55_20220310',
 '16_20230227',
 '116_20220426',
 '58_20220310',
 '21_20230306',
 '37_20230221',
 '46_20230221',
 '98_2022_0426',
 '110_20220426',
 '14_20230301',
 '14_20230301',
 '48_20230227',
 '48_20230227',
 '48_20230227',
 '13_20230221',
 '74_20220426',
 '34_20230227',
 '31_20230227',
 '01_20230221',
 '16_20230227',
 '30_20230227',
 '10_20230227',
 '33_20230221',
 '46_20230221',
 '62_20220324',
 '57_20220310',
 '101_20220310']

In [ ]:
plates = [
    "078_20220920",
    "079_20220920",
    "080_20220920",
    "081_20220920",
    "082_20220920",
    "083_20220920",
    "084_20220920",
    "001_20220920",
    "024_20220920",
    "073_20220920",
    "002_20220920",
    "003_20220920",
    "004_20220920",
    "005_20220920",
    "006_20220920",
    "007_20220920",
    "008_20220920",
    "009_20220920",
    "010_20220920",
    "011_20220920",
    "012_20220920",
    "019_20220920",
    "020_20220920",
    "021_20220920",
    "022_20220920",
    "023_20220920",
    "037_20220920",
    "038_20220920",
    "039_20220920",
    "040_20220920",
    "042_20220920",
    "043_20220920",
    "044_20220920",
    "045_20220920",
    "046_20220920",
    "016_20220921",
    "017_20220921",
    "034_20220921",
    "051_20220920",
    "094_20220921",
    "064_20220921",
    "026_20220921",
    "027_20220921",
    "028_20220921",
    "029_20220921",
    "031_20220921",
    "033_20220921",
    "035_20220921",
    "036_20220921",
    "047_20220920",
    "041_20220920",
    "015_20220921",
    "068_20220921",
    "107_20220921",
    "095_20220921",
    "062_20220921",
    "096_20220921",
    "104_20220921",
    "050_20220920",
    "030_20220921",
    "025_20220921",
    "100_20220921",
    "067_20220921",
    "048_20220920",
    "055_20220921",
    "056_20220921",
    "057_20220921",
    "058_20220921",
    "059_20220921",
    "060_20220921",
    "061_20220921",
    "063_20220921",
    "014_20220921",
    "018_20220921",
    "099_20220921",
    "069_20220921",
    "054_20220920",
    "075_20220920",
    "092_20220921",
    "093_20220921",
    "097_20220921",
    "098_20220921",
    "105_20220921",
    "106_20220921",
    "108_20220921",
    "085_20220921",
    "086_20220921",
    "087_20220921",
    "088_20220921",
    "089_20220921",
    "090_20220921",
    "091_20220921",
    "049_20220920",
    "052_20220920",
    "053_20220920",
    "074_20220920",
    "076_20220920",
    "077_20220920",
    "013_20220921",
    "070_20220921",
    "071_20220921",
    "072_20220921",
    "102_20220921",
    "032_20220921",
    "101_20220921",
    "065_20220921",
    "066_20220921",
    "103_20220921",
]
plates = [
    "_".join((str(int(plate.split("_")[0])), str(int(plate.split("_")[1]))))
    for plate in plates
]

In [ ]:
directory_targ = directory_project
directory_project

In [ ]:
directory_targ

In [ ]:
directory_targ = os.path.join(directory_scratch, "stitch_temp") + "/"

In [ ]:
name_job = f"one_shot_{time_ns()}.sh"
run_launcher(
    "dropbox_downloader.py",
    [directory_targ, name_job, 30, "masker.py"],
    plates,
    "12:00:00",
    name_job=name_job,
    dependency=True,
)

In [ ]:
name_job = f"one_shot_{time_ns()}.sh"
run_launcher(
    "dropbox_downloader.py",
    [directory_targ, name_job, 30, "skeletonizer_agg.py"],
    plates,
    "12:00:00",
    name_job=name_job,
    dependency=True,
)
# run_launcher('dropbox_uploader.py',[directory_targ,name_job],plates,'40:00',dependency=True,name_job = name_job)

In [ ]:
run_launcher(
    "dropbox_uploader.py",
    [directory_targ, name_job],
    plates,
    "24:00:00",
    dependency=True,
    name_job=name_job,
)

In [ ]:
1

In [ ]:
name_job = f"one_shot_{time_ns()}.sh"
name_job = "table_redo"
run_launcher(
    "dropbox_downloader.py",
    [directory_targ, name_job, 30, "post_processer_1.py"],
    plates,
    "24:00:00",
    name_job=name_job,
    dependency=False,
)

In [ ]:
name_job = "strains"

# name_job = f"one_shot_{time_ns()}.sh"

run_launcher(
    "post_processer_1.py",
    [directory_targ, name_job, 15],
    plates,
    "1:00:00",
    dependency=False,
    name_job=name_job,
)

In [ ]:
plates = [
    # '3_20220426',
 # '12_20220502',
 '13_20220422']

In [ ]:
update_analysis_info(directory_targ)

analysis_info = get_analysis_info(directory_targ)
analysis_folders = analysis_info.loc[analysis_info["unique_id"].isin(plates)]

In [ ]:
analysis_folders

In [ ]:

for path in analysis_folders["path_time_plate_info"]:
    if not os.path.exists(os.path.join(directory_targ,path)):
        print(path)

In [ ]:
import sys
from amftrack.util.sys import (
    update_analysis_info,
    get_analysis_info,
)
from amftrack.pipeline.launching.run_super import run_parallel_post
from amftrack.pipeline.functions.post_processing.global_plate import *
from amftrack.pipeline.functions.post_processing.time_plate import *
from amftrack.pipeline.functions.post_processing.global_hypha import *
from amftrack.pipeline.functions.post_processing.area_hulls import *
from amftrack.pipeline.launching.run_super import run_parallel, run_launcher
from amftrack.pipeline.functions.post_processing.exp_plot import *
time = "3:40:00"
directory = directory_targ
# list_f = [get_width_f,get_tot_length_C_f,get_tot_growth_C_f]
# list_f = [get_timestep_anastomosis]
list_f = [
    gets_out_of_ROI,
    get_width_f,
    get_tot_length_C_f,
    get_tot_growth_C_f,
    get_tot_length_pp_f,
    get_tot_growth_pp_f,
    get_timestep_stop_growth,
    get_time_stop_growth,
    get_time_init_growth,
    get_mean_speed_growth,
    get_stop_track,
    get_timestep_anastomosis,
    get_timestep_biological_stop_growth,
]
# # list_f = [get_num_branch]
# # list_f = [gets_out_of_ROI]
list_args = [{}] * len(list_f)
overwrite = True
num_parallel = 4
run_parallel_post(
    "global_hypha_post_process.py",
    list_f,
    list_args,
    [directory_targ, overwrite],
    analysis_folders,
    num_parallel,
    time,
    "global_hypha_post_process",
    cpus=32,
    name_job="strain",
    node="fat",
)

In [ ]:
import sys
from amftrack.util.sys import (
    update_analysis_info,
    get_analysis_info,
)
from amftrack.pipeline.launching.run_super import run_parallel_post
from amftrack.pipeline.functions.post_processing.global_plate import *
from amftrack.pipeline.functions.post_processing.time_plate import *
from amftrack.pipeline.functions.post_processing.global_hypha import *
from amftrack.pipeline.functions.post_processing.area_hulls import *
from amftrack.pipeline.launching.run_super import run_parallel, run_launcher
from amftrack.pipeline.functions.post_processing.exp_plot import *

time = "2:00:00"
directory = directory_targ
max_ind = 20
incr = 100

fs = [
    get_density_in_ring,
    get_density_active_tips_in_ring,
]
# fs = [get_mean_speed_in_ring]
name_job = "agg_sup"
list_f = []
list_args = []

for f in fs:
    list_f += [f] * max_ind

    list_args += [{"incr": incr, "i": i, "rh_only": True} for i in range(max_ind)]
overwrite = False
num_parallel = 6
run_parallel_post(
    "time_plate_post_process.py",
    list_f,
    list_args,
    [directory_targ, overwrite],
    analysis_folders,
    num_parallel,
    time,
    "time_plate_post_process",
    cpus=32,
    name_job=name_job,
    node="fat",
    dependency=True,
)

In [ ]:
time = "3:40:00"
directory = directory_targ
# list_f = [get_width_f,get_tot_length_C_f,get_tot_growth_C_f]
# list_f = [get_timestep_anastomosis]
list_f = [
    gets_out_of_ROI,
    get_width_f,
    get_tot_length_C_f,
    get_tot_growth_C_f,
    get_tot_length_pp_f,
    get_tot_growth_pp_f,
    get_timestep_stop_growth,
    get_time_stop_growth,
    get_time_init_growth,
    get_mean_speed_growth,
    get_stop_track,
    get_timestep_anastomosis,
    get_timestep_biological_stop_growth,
]
# # list_f = [get_num_branch]
# # list_f = [gets_out_of_ROI]
list_args = [{}] * len(list_f)
overwrite = True
num_parallel = 32
run_parallel_post(
    "global_hypha_post_process.py",
    list_f,
    list_args,
    [directory, overwrite],
    analysis_folders,
    num_parallel,
    time,
    "global_hypha_post_process",
    cpus=32,
    name_job=name_job,
    node="fat",
)

In [ ]:
plates

In [ ]:
name_job = "node_id"

run_launcher(
    "node_identifier.py",
    [directory_targ, name_job, -1],
    plates,
    "24:00:00",
    dependency=True,
    name_job=name_job,
)

In [ ]:
plates = ['401_20231123', '395_202311201']

In [ ]:
name_job = "Loreto"

run_launcher(
    "stitcher.py",
    [directory_targ, name_job, -1],
    plates,
    "5:00:00",
    dependency=False,
    name_job=name_job,
)

In [ ]:
# name_job = "recompute"

# name_job = f"one_shot_{time_ns()}.sh"
name_job = "one_shot_1675411679561103361"
run_launcher(
    "analysis_uploader.py",
    [directory_targ, name_job, -1],
    plates,
    "12:00:00",
    dependency=True,
    name_job=name_job,
)

In [ ]:
from amftrack.util.dbx import upload_folder

update_analysis_info(directory_targ)
analysis_info = get_analysis_info(directory_targ)
analysis_folders = analysis_info.loc[analysis_info["unique_id"].isin(plates)]
dir_drop = "DATA/PRINCE"

for index, row in analysis_folders.iterrows():
    folder = row["folder_analysis"]
    id_unique = row["unique_id"]
    path = os.path.join(directory_targ, folder)
    target_drop = f"/{dir_drop}/{id_unique}/{folder}"
    print(dir_drop)
    upload_folder(path, target_drop, delete=False)

In [ ]:
name_job = f"one_shot_{time_ns()}.sh"
run_launcher(
    "width_extracter.py",
    [directory_targ, name_job, 20],
    plates,
    "20:00",
    dependency=False,
    name_job=name_job,
)

In [ ]:
update_plate_info(directory_targ, local=True)
all_folders = get_current_folders(directory_targ, local=True)
# folders =all_folders.loc[all_folders["unique_id"].isin(plates)]
# folders = folders.loc[folders["/Analysis/skeleton_pruned_compressed.mat"]==True]

In [ ]:
all_folders["unique_id"].unique()

In [ ]:
folders = all_folders.loc[all_folders["unique_id"].isin(plates)]
folders = folders.loc[folders["/Analysis/skeleton_compressed.mat"] == True]
folders = folders.sort_values("datetime")
folders

In [ ]:
limit = 200
folders = folders.sort_values("datetime")
# folders = folders.iloc[:5]
corrupted_rotation = folders.loc[
    ((folders["/Analysis/transform.mat"] == False))
    & (folders["/Analysis/transform_corrupt.mat"])
]["folder"]

folder_list = list(folders["folder"])
folder_list.sort()
indexes = [folder_list.index(corrupt_folder) for corrupt_folder in corrupted_rotation]
indexes = [index for index in indexes if index < limit]
indexes.sort()
indexes += [limit]
indexes = [limit]


In [ ]:
folders

In [ ]:
indexes

In [ ]:
plates = ['12_20220502', '229_20230330']

In [ ]:
folders = all_folders.loc[all_folders["unique_id"].isin(plates)]
folders = folders.loc[folders["/Analysis/nx_graph_pruned_width.p"] == True]
args = [directory_targ]
num_parallel = 128

for unique_id in plates:
    select = folders.loc[folders["unique_id"] == unique_id]
    time = "12:00:00"
    run_parallel(
        "track_nodes.py",
        args,
        select,
        num_parallel,
        time,
        "track_node",
        cpus=128,
        node="fat",
        name_job=name_job,
    )

In [ ]:
update_analysis_info(directory_targ)
analysis_info = get_analysis_info(directory_targ)
analysis_folders = analysis_info.loc[analysis_info["unique_id"].isin(plates)]

In [ ]:
analysis_folders

In [ ]:
name_job = "agg_clarum"
folders = folders.loc[folders["/Img/TileConfiguration.txt.registered"] == True]
num_parallel = 100
time = "6:00:00"
hyph_width = 30
perc_low = 85
perc_high = 99.5
minlow = 10
minhigh = 70

args = [hyph_width, perc_low, perc_high, minlow, minhigh,directory_targ]
run_parallel(
    "extract_skel_2.py",
    args,
    folders,
    num_parallel,
    time,
    "skeletonization",
    cpus=128,
    node="fat",
    name_job=name_job,
)

In [ ]:
folders = all_folders.loc[all_folders["unique_id"].isin(plates)]
folders = folders.loc[folders["/Analysis/nx_graph_pruned_labeled.p"] == True]


In [ ]:
for plate in plates:
    folders = all_folders.loc[all_folders["unique_id"]==plate]
    folders1 = folders.loc[folders["/Analysis/nx_graph_pruned_labeled.p"] == True]
    folders2 = folders.loc[folders["/Analysis/skeleton_compressed.mat"] == True]  
    print(plate,len(folders1),len(folders2),folders1['folder'].sort_values().iloc[0],folders2['folder'].sort_values().iloc[0])
    # if plate == '13_20220422':
    #     break

In [ ]:
folders1['folder'].sort_values(),folders2['folder']

In [ ]:
name_job = "empty"
dir_drop = "DATA/PRINCE"
delete = True
run_parallel_transfer(
    "toward_drop.py",
    [dir_drop, delete],
    folders,
    50,
    "24:00:00",
    "staging",
    cpus=1,
    node="staging",
    name_job=name_job,
)

In [ ]:
plates = [plate for plate in all_folders["unique_id"].unique() if plate not in plates]

In [ ]:
from amftrack.pipeline.launching.run_super import (
    run_parallel,
    run_launcher,
    run_parallel_all_time,
)
args = [directory_targ]
num_parallel = 32
name_job = "agg_rerun"
time = "6:00:00"
run_parallel_all_time(
    "make_labeled_graphs.py",
    args,
    folders,
    num_parallel,
    time,
    "make_graphs",
    cpus=32,
    node="thin",
    dependency=True,
    name_job=name_job,
)

In [ ]:
all_folders["unique_id"].unique()

In [ ]:
folders.loc[folders["/Analysis/transform.mat"] == True]

In [ ]:
from amftrack.pipeline.launching.run_super import run_parallel, run_launcher

name_job = "malin"
stage = 30
args = [directory_targ]
for plate in plates:
    select = folders.loc[folders["unique_id"] == plate]
    num_parallel = 32
    time = "10:00"
    args = [directory_targ]
    run_parallel(
        "realign.py",
        args,
        select,
        num_parallel,
        time,
        "create_realign",
        cpus=32,
        node="fat",
        name_job=name_job,
    )

run_launcher(
    "graph_extracter.py",
    [directory_targ, name_job, stage - 1],
    plates,
    "3:00:00",
    dependency=True,
    name_job=name_job,
)

In [ ]:
name_job = "rerun"
run_launcher(
    "post_processer_1.py",
    [directory_targ, name_job, 10],
    plates,
    "3:00:00",
    dependency=True,
    name_job=name_job,
)

In [ ]:
from amftrack.util.dbx import upload_folder

update_analysis_info(directory_targ)
analysis_info = get_analysis_info(directory_targ)
analysis_folders = analysis_info.loc[analysis_info["unique_id"].isin(plates)]
dir_drop = "DATA/PRINCE"

for index, row in analysis_folders.iterrows():
    folder = row["folder_analysis"]
    id_unique = row["unique_id"]
    path = os.path.join(directory_targ, folder)
    target_drop = f"/{dir_drop}/{id_unique}/{folder}"
    print(dir_drop)
    upload_folder(path, target_drop, delete=False)

In [ ]:
from amftrack.pipeline.functions.post_processing.exp_plot import *
from amftrack.pipeline.launching.run_super import run_parallel_post

name_job = "malin"
time = "1:00:00"
list_f = [plot_tracking]
update_analysis_info(directory_targ)
analysis_info = get_analysis_info(directory_targ)
analysis_folders = analysis_info.loc[analysis_info["unique_id"].isin(plates)]
directory = directory_targ
list_args = [[]] * len(list_f)
overwrite = True
num_parallel = 100
run_parallel_post(
    "exp_plot.py",
    list_f,
    list_args,
    [directory, overwrite],
    analysis_folders,
    num_parallel,
    time,
    "global_plate_post_process",
    cpus=128,
    name_job=name_job,
    node="fat",
)

In [ ]:
from amftrack.pipeline.functions.post_processing.time_hypha import *
import pandas as pd
import os

update_analysis_info(directory_targ)
analysis_info = get_analysis_info(directory_targ)
analysis_folders = analysis_info.loc[analysis_info["unique_id"].isin(plates)]
list_f = [
    get_pos_x,
    get_pos_y,
    get_time_since_begin_exp,
    get_distance_final_pos,
    get_timedelta,
    get_time_since_start,
    get_speed,
    get_timestep,
    get_timestep_init,
    get_time_init,
    get_degree,
    get_width_tip_edge,
    get_width_root_edge,
    get_width_average,
    get_has_reached_final_pos,
    get_in_ROI,
]
# list_f = [local_density,local_density,local_density]
# list_f = [get_time_since_begin_exp]
# list_f = [get_width_tip_edge, get_width_root_edge]
list_args = [{}] * len(list_f)
# list_args= [[500],[1000],[2000]]+[[]]
# list_args= [[500]]
overwrite = True
load_graphs = True
num_parallel = 32
time = "10:00"
for index, row in analysis_folders.iterrows():
    folder = row["folder_analysis"]
    path_time_plate_info = row["path_time_plate_info"]
    plate = row["Plate"]
    num_cpus = 32
    if os.path.isfile(f"{directory_targ}{path_time_plate_info}"):
        whole_plate_info = pd.read_json(
            f"{directory_targ}{path_time_plate_info}", convert_dates=True
        ).transpose()
        whole_plate_info.index.name = "t"
        whole_plate_info.reset_index(inplace=True)
        run_parallel_post(
            "time_hypha_post_process.py",
            list_f,
            list_args,
            [directory_targ, overwrite, load_graphs],
            whole_plate_info,
            num_parallel,
            time,
            "time_hypha_post_process",
            cpus=num_cpus,
            name_job=name_job,
            node="fat",
        )

In [ ]:
from amftrack.pipeline.launching.run_super import (
    run_parallel,
    run_launcher,
    run_parallel_all_time,
)

folders = all_folders.loc[all_folders["unique_id"].isin(plates)]
folders = folders.loc[folders["/Analysis/nx_graph_pruned_labeled.p"] == True]
args = [directory_targ]
num_parallel = 32
time = "1:00:00"
limit = 200
version = 1
suffix = "_labeled"
lim_considered = 10
args = [directory_targ, limit, version, suffix, lim_considered]
run_parallel_all_time(
    "hyphae_extraction.py",
    args,
    folders,
    num_parallel,
    time,
    "hyphae",
    cpus=32,
    node="fat",
    name_job="test",
)

In [ ]:
folders = folders.loc[folders["/Analysis/nx_graph_pruned_labeled.p"] == True]
len(folders)

In [ ]:
len(list(set(folders["unique_id"].values)))

In [ ]:
stage = 20
run_launcher(
    "post_processer_1.py",
    [directory_targ, name_job, stage - 1],
    plates,
    "20:00",
    dependency=True,
    name_job="test",
)

In [ ]:
update_analysis_info(directory_targ)
analysis_info = get_analysis_info(directory_targ)
analysis_folders = analysis_info.loc[analysis_info["unique_id"].isin(plates)]

In [ ]:
analysis_folders

In [ ]:
# name_job = f"one_shot_{time_ns()}.sh"

from amftrack.pipeline.launching.run_super import run_parallel_post
from amftrack.pipeline.functions.post_processing.global_plate import *
from amftrack.pipeline.functions.post_processing.time_plate import *
from amftrack.pipeline.functions.post_processing.global_hypha import *
from amftrack.pipeline.functions.post_processing.area_hulls import *
from amftrack.pipeline.launching.run_super import run_parallel, run_launcher
from amftrack.pipeline.functions.post_processing.exp_plot import *

time = "24:00:00"
directory = directory_targ
max_ind = 20
incr = 100
fs = [
    get_density_in_ring,
    get_density_anastomose_in_ring,
    # get_density_branch_rate_in_ring,
    # get_density_stop_rate_in_ring,
    get_density_active_tips_in_ring,
]
# fs = [get_mean_speed_in_ring]

list_f = []
list_args = []

for f in fs:
    list_f += [f] * max_ind

    list_args += [{"incr": incr, "i": i, "rh_only": True} for i in range(max_ind)]
overwrite = False
num_parallel = 30
run_parallel_post(
    "time_plate_post_process.py",
    list_f,
    list_args,
    [directory, overwrite],
    analysis_folders,
    num_parallel,
    time,
    "time_plate_post_process",
    cpus=32,
    name_job=name_job,
    node="fat",
    dependency=True,
)
run_launcher(
    "analysis_uploader.py",
    [directory_targ, name_job],
    plates,
    "12:00:00",
    dependency=True,
    name_job=name_job,
)

In [ ]:
from amftrack.pipeline.launching.run_super import run_parallel, run_launcher

name_job = "malin"

num_parallel = 100
time = "20:00"
args = [directory_targ]
run_parallel(
    "compress_image.py",
    args,
    folders,
    num_parallel,
    time,
    "compress",
    cpus=128,
    node="fat",
    name_job=name_job,
)

In [ ]:
from amftrack.pipeline.launching.run_super import run_parallel, run_launcher

num_parallel = 50
time = "12:00:00"
hyph_width = 30
perc_low = 93
perc_high = 99.5
args = [hyph_width, perc_low, perc_high, directory_targ]
run_parallel(
    "extract_skel_2.py",
    args,
    folders,
    num_parallel,
    time,
    "skeletonization",
    cpus=128,
    node="fat",
    name_job=name_job,
)

In [ ]:
from amftrack.pipeline.launching.run_super import run_parallel, run_launcher

num_parallel = 100
time = "20:00"
args = [directory_targ]
run_parallel(
    "detect_blob.py",
    args,
    folders,
    num_parallel,
    time,
    "detect_blob",
    cpus=128,
    node="fat",
    name_job=name_job,
)

In [ ]:
name_job = "malin"

run_launcher(
    "masker.py",
    [directory_targ, name_job, 30],
    plates,
    "3:00:00",
    dependency=True,
    name_job=name_job,
)

In [ ]:
from amftrack.pipeline.launching.run_super import run_parallel_all_time, run_launcher

folders = all_folders.loc[all_folders["unique_id"].isin(plates)]
folders = folders.loc[folders["/Img/TileConfiguration.txt.registered"] == True]
num_parallel = 50
time = "10:00"
args = []
run_parallel_all_time(
    "make_video_stitched.py",
    args,
    folders,
    num_parallel,
    time,
    "make_video",
    cpus=32,
    node="fat",
    dependency=False,
    name_job="video",
)

In [ ]:
plates = [
    "105_20220817",
    "77_20220808",
    "71_20220817",
    "100_20220812",
    "117_20220815",
    "99_20220815",
    "75_20220808",
    "226_20220812",
]

In [ ]:
folders = all_folders.loc[all_folders["Plate"].isin(["32"])]

In [ ]:
from amftrack.pipeline.launching.run_super import *

num_parallel = 32
time = "8:00:00"
limit = 200
version = 1
labeled = True
args = [directory_targ, limit, version, labeled]
run_parallel_all_time(
    "hyphae_extraction.py",
    args,
    folders,
    num_parallel,
    time,
    "hyphae",
    cpus=32,
    node="fat",
)

In [ ]:
from amftrack.pipeline.launching.run_super import run_parallel, run_launcher

for plate in plates:
    select = folders.loc[folders["Plate"] == plate]
    num_parallel = 128
    time = "12:00:00"
    thresh = 10000  # For R. irregularis, thresh 10000 is good. For Aggregatum, higher may be necessary
    args = [thresh, directory_targ]
    run_parallel(
        "final_alignment.py",
        args,
        select,
        num_parallel,
        time,
        "realign",
        cpus=128,
        node="fat",
        name_job=name_job,
    )

In [ ]:
plates

In [ ]:
select

In [ ]:
i = 0
run_info = folders
folder_list = list(run_info["folder"])
folder_list.sort()

dates_datetime_chosen = folder_list[i : i + 2]
print("========")
print(f"Matching plate {dates_datetime_chosen[0]} at dates {dates_datetime_chosen}")
print("========")
dates = dates_datetime_chosen

In [ ]:
from amftrack.pipeline.launching.run_super import run_parallel_post
from amftrack.pipeline.functions.post_processing.global_plate import *
from amftrack.pipeline.functions.post_processing.time_plate import *
from amftrack.pipeline.functions.post_processing.global_hypha import *
from amftrack.pipeline.functions.post_processing.area_hulls import *
from amftrack.pipeline.launching.run_super import run_parallel, run_launcher
from amftrack.pipeline.functions.post_processing.exp_plot import *

update_analysis_info(directory_targ)
analysis_info = get_analysis_info(directory_targ)
analysis_folders = analysis_info.loc[analysis_info["unique_id"].isin(plates)]
directory = directory_targ

In [ ]:
time = "3:40:00"
directory = directory_targ
# list_f = [get_width_f,get_tot_length_C_f,get_tot_growth_C_f]
# list_f = [get_timestep_anastomosis]
list_f = [
    gets_out_of_ROI,
    get_width_f,
    get_tot_length_C_f,
    get_tot_growth_C_f,
    get_tot_length_pp_f,
    get_tot_growth_pp_f,
    get_timestep_stop_growth,
    get_time_stop_growth,
    get_time_init_growth,
    get_mean_speed_growth,
    get_stop_track,
    get_timestep_anastomosis,
    get_timestep_biological_stop_growth,
]
# list_f = [get_num_branch]
# list_f = [gets_out_of_ROI]
list_args = [{}] * len(list_f)
overwrite = False
num_parallel = 32
run_parallel_post(
    "global_hypha_post_process.py",
    list_f,
    list_args,
    [directory, overwrite],
    analysis_folders,
    num_parallel,
    time,
    "global_hypha_post_process",
    cpus=32,
    name_job="a",
    node="fat",
)

In [ ]:
import sys
from amftrack.util.sys import (
    update_analysis_info,
    get_analysis_info,
)
from amftrack.pipeline.launching.run_super import run_parallel_post
from amftrack.pipeline.functions.post_processing.time_hypha import *
from amftrack.pipeline.launching.run_super import run_parallel, run_launcher
import pandas as pd
import os

list_f = [
    get_pos_x,
    get_pos_y,
    get_time_since_begin_exp,
    get_distance_final_pos,
    get_timedelta,
    get_time_since_start,
    get_speed,
    get_timestep,
    get_timestep_init,
    get_time_init,
    get_degree,
    get_width_tip_edge,
    get_width_root_edge,
    get_width_average,
    has_reached_final_pos,
    in_ROI,
]
# list_f = [local_density,local_density,local_density]
# list_f = [get_time_since_begin_exp]
# list_f = [get_width_tip_edge, get_width_root_edge]
list_args = [{}] * len(list_f)
# list_args= [[500],[1000],[2000]]+[[]]
# list_args= [[500]]
overwrite = False
load_graphs = True
num_parallel = 32
time = "12:00:00"
for index, row in analysis_folders.iterrows():
    folder = row["folder_analysis"]
    path_time_plate_info = row["path_time_plate_info"]
    plate = row["Plate"]
    num_cpus = 32
    if os.path.isfile(f"{directory_targ}{path_time_plate_info}"):
        whole_plate_info = pd.read_json(
            f"{directory_targ}{path_time_plate_info}", convert_dates=True
        ).transpose()
        whole_plate_info.index.name = "t"
        whole_plate_info.reset_index(inplace=True)
        run_parallel_post(
            "time_hypha_post_process.py",
            list_f,
            list_args,
            [directory_targ, overwrite, load_graphs],
            whole_plate_info,
            num_parallel,
            time,
            "time_hypha_post_process",
            cpus=num_cpus,
            name_job=name_job,
            node="fat",
        )